In [1]:
# Imports
from pprint import pprint
import json
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians

In [2]:
gps_data = []
with open('gps_dump.json') as file:
    for line in file:
        parts = line.split(' ',1) 
        if '/gps/rmc' in parts[0]:
            gps_data.append(parts)
# pprint(gps_data[:5])

In [3]:
data_gps = []
import pynmea2
for item in gps_data:
    data_list = item[1].split(',')
    msg = pynmea2.parse(item[1])
    try:
        data_gps.append((item[0].replace('owntracks/','').replace('/gps/rmc',''),
                        data_list[9],
                        msg.timestamp,
                        data_list[2],
                        msg.latitude,
                        msg.longitude))
    except:
        pass
df_gps = pd.DataFrame(data=data_gps, columns=['vehicle','date','time','active or void','lat','lon'])
# df_gps.to_csv(path_or_buf='gps_data.csv',index=False)

In [4]:
trams_with_connection_loss = df_gps[df_gps['active or void'] == 'V']
# display(trams_with_connection_loss)
trams = df_gps.groupby('vehicle')
dict_trams = {}

for name,group in trams:
    tram_locations = []
    
    conn = None
    pre_loc = None
    for idx, row in group.iterrows():
#         print(row)
        if conn is None:
            conn = row['active or void']
            pre_loc = (row['lat'], row['lon'])
        elif conn == 'A' and row['active or void'] == 'V':
            tram_locations.append(pre_loc)
            conn = row['active or void']
        elif conn == row['active or void'] and conn == 'A':
            conn = row['active or void']
            pre_loc = (row['lat'], row['lon'])
        elif conn == 'V' and row['active or void'] == 'A':
            conn = row['active or void']
            pre_loc = (row['lat'], row['lon'])
            tram_locations.append(pre_loc)
        else:
            pass
    if  tram_locations:
        dict_trams[name] = tram_locations
        
# pprint(dict_trams)
# with open('tram_lost_locations.json', 'w') as json_file:  
#     json.dump(dict_trams, json_file)


In [9]:
# locations where connection where lost and where regained
tram_locations = []
for name,group in trams:
    conn = None
    pre_loc = None
    for idx, row in group.iterrows():
#         print(row)
        if conn is None:
            conn = row['active or void']
            pre_loc = (row['lat'], row['lon'])
        elif conn == 'A' and row['active or void'] == 'V':
            pre_loc = (0,) + pre_loc
            tram_locations.append(pre_loc)
            conn = row['active or void']
        elif conn == row['active or void'] and conn == 'A':
            conn = row['active or void']
            pre_loc = (row['lat'], row['lon'])
        elif conn == 'V' and row['active or void'] == 'A':
            conn = row['active or void']
            pre_loc = (1, row['lat'], row['lon'])
            tram_locations.append(pre_loc)
        else:
            pass
# pprint(tram_locations)
with open('tram_lost_locations.csv', 'w') as f:  
    for item in tram_locations:
        f.write(str(item[0])+','+str(item[1])+','+str(item[2])+'\n')

In [5]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # calculate distance between two lat,lon locations
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
#     print(distance)
    return distance


active_trams = df_gps[df_gps['active or void'] == 'A']
# active_trams.sort_values(by=['time','lat','lon'])


In [ ]:
# calculate the distance between data points - very costly...
active_trams.set_index('time', inplace=True)
active_trams = active_trams.drop(columns=['date','active or void'])

sLength = len(active_trams['vehicle'])
active_trams['distance to prior']= np.zeros(sLength)

pre_row = None
for idx, row in active_trams.iterrows():
    if pre_row is None:
        pre_row = row
    else:
        distance = calculate_distance(pre_row['lat'], pre_row['lon'], row['lat'], row['lon'])
        active_trams.set_value(idx,'distance to prior',distance)
        pre_row = row
display(active_trams)

C:\Users\natha\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [7]:
# prepare vehicle, lat and lon for demo of queuing algorithm
new_locs = df_gps.drop(columns=['date','time','active or void'])
new_locs.to_csv(path_or_buf='new_locations.csv',index=False)

In [5]:
print(len(df_gps['vehicle'].unique()))

288
